In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Vector creation

In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

/home/aakash/miniconda3/envs/gsoc/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Sentences we want to encode. Example:
sentence = ['Trying out the onto-embeddings notebook']

import time
s = time.time()
#Sentences are encoded by calling model.encode()
embedding = model.encode(sentence)
e = time.time()
print(f"{e-s} seconds taken to encode one sentence")

2.114689350128174 seconds taken to encode one sentence


In [4]:
embedding.shape

(1, 384)

In [5]:
from SPARQLWrapper import SPARQLWrapper, JSON

NS_RESOURCE = 'http://dbpedia.org/resource/'
NS_RESOURCE_LEN = len(NS_RESOURCE)

NS_ONTOLOGY = 'http://dbpedia.org/ontology/'
NS_ONTOLOGY_LEN = len(NS_ONTOLOGY)

In [7]:
def retrieve_tbox(lang='en', offset=0):
    sparql = SPARQLWrapper('http://dbpedia.org/sparql')
    query = f"""
    SELECT ?uri ?label {{
      ?uri a ?type ; rdfs:label ?label .
      values(?type) {{ (owl:Class) (rdf:Property) }}
      filter(lang(?label) = '{lang}' && regex(?uri, "http://dbpedia.org/ontology/"))
    }} LIMIT 10000 OFFSET {offset}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    tbox = {}
    for result in results['results']['bindings']:
        uri = result['uri']['value']
        label = result['label']['value']
        if label not in tbox:
            tbox[label] = set()
        tbox[label].add(uri)
    return tbox

In [8]:
offset = 0
tbox = {}
while True:
    tbox_chunk = retrieve_tbox(lang='en', offset=offset)
    if len(tbox_chunk) == 0:
        break
    offset += 10000
    for k, v in tbox_chunk.items():
        if k not in tbox:
            tbox[k] = set()
        tbox[k] = tbox[k].union(v)
len(tbox)

3520

In [9]:
labels = [l.replace('\n', ' ') for l in tbox]

In [10]:
embeddings = model.encode(labels)

with open('dbpedia-ontology.vectors', 'w', encoding='utf-8') as f_out:
    f_out.write(f"{len(labels)} {len(embeddings[0])}\n")
    for label, embedding in zip(labels, embeddings):
        f_out.write(f"{label.replace(' ', '_')} {' '.join([str(x) for x in embedding])}\n")

Batches:   0%|          | 0/110 [00:00<?, ?it/s]

In [ ]:
model_ = SentenceTransformer('paraphrase-MiniLM-L6-v2')

### Query Pipeline

In [11]:
from gensim.models import KeyedVectors

In [12]:
w2v_model = KeyedVectors.load_word2vec_format('dbpedia-ontology.vectors', binary=False)

In [13]:
import pandas as pd

def to_uri(label):
    return list(filter(lambda x: 'A' <= x[NS_ONTOLOGY_LEN : NS_ONTOLOGY_LEN+1] <= 'z', tbox[label]))

def ontosim_search(term):
    result = w2v_model.most_similar(positive=model.encode([term]), topn=5)
    out = []
    for label, score in result:
        out.append({'label': label.replace('_', ' '), 'score': score})
    df = pd.DataFrame(out)
    df.insert(0, 'URIs', df['label'].map(to_uri))
    return df

In [14]:
df = ontosim_search('vice-president')
with pd.option_context('max_colwidth', None, 'display.width', 200):
    print(df)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                                                                                             URIs                label     score
0          [http://dbpedia.org/ontology/VicePresident, http://dbpedia.org/ontology/vicePresident]       vice president  0.981270
1                                                        [http://dbpedia.org/ontology/viceLeader]          vice leader  0.900534
2                  [http://dbpedia.org/ontology/president, http://dbpedia.org/ontology/President]            president  0.750471
3                                                    [http://dbpedia.org/ontology/viceChancellor]      vice chancellor  0.730483
4  [http://dbpedia.org/ontology/VicePrimeMinister, http://dbpedia.org/ontology/vicePrimeMinister]  vice prime minister  0.683350


In [21]:
ontosim_search("on the bank of river")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,URIs,label,score
0,"[http://dbpedia.org/ontology/river, http://dbp...",river,0.849747
1,[http://dbpedia.org/ontology/BodyOfWater],body of water,0.683419
2,[http://dbpedia.org/ontology/riverMouth],river mouth,0.682989
3,[http://dbpedia.org/ontology/Canal],canal,0.653355
4,[http://dbpedia.org/ontology/water],water,0.620236
